In [1]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from signal_dataset import SignalDataset
from sklearn.model_selection import train_test_split
import re
from pathlib import Path
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y
import yaml

In [20]:
sr = 1562500
signal_data_dir = "/home/petr/Documents/Motor_project/AE_PETR_motor/"
bin_setup = [{"label": i.stem, "channels": len(list(i.glob('*.bin'))), "interval": [0, 15 * sr], "bin_path": list(i.glob('*.bin'))[0]} for i in
             Path(signal_data_dir).glob('WUP*') if re.search(r'\d$', i.stem)]

sd = SignalDataset(step=1000, window_size=1000, bin_setup=bin_setup, device="cpu", source_dtype="float32", sklearn=True)

In [4]:
from torch.utils.data import random_split
train_data, test_data = random_split(sd, [0.8, 0.2])

In [ ]:
from sklearn.linear_model import LinearRegression


class SklearnModel(BaseEstimator, ClassifierMixin):
    def __init__(self, control_center: Path):
        self.control_center = control_center
        self._load_yaml(control_center)
        self.model = self._build_model()
    def _load_yaml(self, control_center):
        with control_center.open(mode="r") as yaml_file:
            self.config = yaml.load(yaml_file, Loader=yaml.SafeLoader)
            self.model_config = self.config["model"]
    def _build_model(self):
        match self.model_config["class"]:
            case "SVM":
                return SVC(kernel=self.model_config["kernel"], C=self.model_config["C"])
            case "LinearRegression":
                return LinearRegression()
            case "RandomForest":
                return RandomForestClassifier()
            case "DummyClassifier":
                return DummyClassifier()

    def train_model(self, train_data):
        X_train = np.asarray([data[0] for data in train_data]) 
        y_train = np.asarray([data[1] for data in train_data]) 

        
        self.classes_ = np.unique(y_train)
        self.model.fit(X_train, y_train)
    
    def predict(self, test_data):
        X_test = np.asarray([data[0] for data in test_data]) 
        self.model.predict(X_test)